### 함수 모듈화

In [31]:
!pip install --upgrade pip
!git clone https://github.com/kakaobrain/pororo.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Cloning into 'pororo'...
remote: Enumerating objects: 2119, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2119 (delta 11), reused 22 (delta 8), pack-reused 2090
Receiving objects: 100% (2119/2119), 13.09 MiB | 4.96 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.


In [32]:
cd pororo

/content/pororo


In [33]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/pororo
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.8/748.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 117.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 119.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 109.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 23.3 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
from konlpy.tag import Okt
from tqdm import tqdm

In [4]:
okt = Okt()

In [5]:
def tokenizing(soynlp_tokenizer, df):
  '''
  df['title']과 토크나이저를 넣으면 토큰화된 리스트 반환하는 함수
  soynlp_tokenizer = 토크나이저
  df = df['title'] 형태
  '''
  # df[title]을 리스트로 저장
  menu = df.tolist()
  
  # 리스트화된 메뉴명을 토크나이징
  menu = [soynlp_tokenizer.nouns(i) for i in tqdm(menu)]

  # 리스트 내 리스트 형식 제거
  menu = [j for i in menu for j in i]

  return menu

In [6]:
from collections import Counter
def token_counting(menu):
  '''
  menu 리스트를 넣으면 빈도 내림차순으로 정렬된 튜플 형태의 리스트를 반환하는 함수
  '''
  menu_count = Counter(menu)
  items = menu_count.items()
  sorted_items = sorted(items, key=lambda x: x[1], reverse=True) 


  return sorted_items

In [7]:
def get_items_freq(menu, cut_num, cut_len=1):
  '''
  토큰 리스트를 넣으면 특정 빈도 이상, 특정 길이 이상의 토큰을 반환하는 함수
  menu = 토큰 리스트
  cut_num = 빈도 제한 (이상)
  cut_len = 길이 제한 (이상)
  '''
  lst = token_counting(menu)
  lst = [v for v, i in lst if i >= cut_num and len(v) >= cut_len]
  lst = sorted(lst, key=lambda x: len(x), reverse=True)
  return lst

In [8]:
def get_items_precent(menu, cut_percent=10, cut_len=1):
  '''
  토큰 리스트를 넣으면 특정 비율 이상, 특정 길이 이상의 토큰을 반환하는 함수
  menu = 토큰 리스트
  cut_percent = 빈도 제한 (상위 n퍼센트)
  cut_len = 길이 제한 (이상)
  '''
  lst = token_counting(menu)
  lst_cut = [v for v, i in lst if len(v)>=cut_len]

  # 상위 cut_percent %의 토큰만 남기기기
  cut_percent = 100 / cut_percent
  cut_line = int(len(lst) / cut_percent)


  lst = lst_cut[:cut_line]
  lst = sorted(lst, key=lambda x: len(x), reverse=True)
  return lst

In [49]:
import numpy as np

def categorizing(tokens, df, feature):
  df['cat'] = None
  df['is_cat'] = False

  for token in tqdm(tokens):
    for i, v in enumerate(df[feature]):
      if v.endswith(token) and df.loc[i, 'cat'] == None:
        df.loc[i, 'cat'] = token
        df.loc[i, 'is_cat'] = True
  return df

In [19]:
df = pd.read_csv('/content/menu_concated_v1.4', index_col=0)

In [20]:
tokens = tokenizing(okt, df['menu'])
tokens[:10]

100%|██████████| 51437/51437 [00:46<00:00, 1102.26it/s]


['정식', '짜장면', '짬뽕', '우동', '울면', '간짜장', '사천짜장', '삼선짜장', '유니짜장', '삼선']

In [21]:
token_counting(tokens)

[('치즈', 1836),
 ('치킨', 1403),
 ('세트', 1367),
 ('추가', 1298),
 ('라떼', 977),
 ('볶음', 850),
 ('밥', 824),
 ('새우', 815),
 ('크림', 796),
 ('탕', 791),
 ('튀김', 789),
 ('피자', 718),
 ('구이', 700),
 ('티', 684),
 ('정식', 662),
 ('불고기', 661),
 ('살', 660),
 ('생', 584),
 ('양념', 560),
 ('샐러드', 551),
 ('듬', 510),
 ('해물', 473),
 ('파스타', 470),
 ('초코', 464),
 ('김치', 455),
 ('갈비', 450),
 ('돈까스', 432),
 ('전골', 424),
 ('한우', 423),
 ('떡볶이', 419),
 ('밀크', 404),
 ('돼지', 402),
 ('우동', 400),
 ('짬뽕', 397),
 ('이드', 392),
 ('스테이크', 386),
 ('감자', 385),
 ('볶음밥', 382),
 ('레몬', 382),
 ('김밥', 377),
 ('소고기', 349),
 ('딸기', 337),
 ('야채', 321),
 ('닭', 321),
 ('스무디', 320),
 ('샌드위치', 317),
 ('사리', 316),
 ('매콤', 316),
 ('커피', 312),
 ('스페셜', 312),
 ('고기', 310),
 ('찜', 308),
 ('만두', 305),
 ('허니', 301),
 ('마늘', 301),
 ('간장', 301),
 ('버섯', 300),
 ('통', 298),
 ('마리', 298),
 ('오징어', 294),
 ('갈릭', 283),
 ('오리', 280),
 ('초밥', 278),
 ('철판', 274),
 ('수제', 273),
 ('삼겹', 273),
 ('베이컨', 272),
 ('블랙', 270),
 ('족발', 269),
 ('변경', 267),
 ('전복', 26

In [42]:
token_selected = get_items_freq(tokens, cut_num=5, cut_len=2)

In [50]:
token_selected[:10]

['소프트아이스크림',
 '까베르네쇼비뇽',
 '까베르네소비뇽',
 '함박스테이크',
 '아스파라거스',
 '프렌치토스트',
 '프렌치프라이',
 '상어지느러미',
 '아메리카노',
 '아이스크림']

In [51]:
categorizing(token_selected, df, 'menu')

100%|██████████| 2514/2514 [01:04<00:00, 39.04it/s]


,식당명,메뉴명,메뉴가격,menu,cat,is_cat
0,신풍식당,정식,5000,정식,정식,True
1,부창루,짜장면,5000,짜장면,짜장면,True
2,부창루,짬뽕,6000,짬뽕,짬뽕,True
3,부창루,우동,6000,우동,우동,True
4,부창루,울면,6000,울면,울면,True
...,...,...,...,...,...,...
51432,차이나프로,철판통후추소고기(대/4~5인분),70000,철판통후추소고기,소고기,True
51433,차이나프로,삼선해물누룽지탕(대/4~5인분),50000,삼선해물누룽지탕,None,False
51434,차이나프로,비풍당새우(대/4~5인분/저녁메뉴),55000,비풍당새우,새우,True
51435,차이나프로,파인새우(대/4~5인분),48000,파인새우,새우,True
